In [2]:
import streamlit as st
import pandas as pd

import shutil
import os

In [3]:
url = 'https://raw.githubusercontent.com/Alejandro-q99/Financial-APP/main/app/Data/bonos.csv'
df = pd.read_csv(url)
df

,diferencia,dq,indice,md,paridad,pq,precio,qp,ticker,tir,ttir,uptir,volumen,vt
0,NaN,43.0,Badlar,0.19,97.2%,31.8,115.20,28%,BDC24,224.3%,99.0%,15.8%,0.0,118.5
1,NaN,63.0,Badlar,0.25,105.8%,33.3,118.80,28%,TB24,161.2%,99.0%,10.0%,0.0,112.3
2,NaN,92.0,Badlar,0.23,89.7%,34.3,91.39,37%,PBY24,315.9%,99.0%,36.5%,0.0,101.9
3,NaN,57.0,Badlar,0.38,84.5%,32.8,96.50,34%,PBA25,295.0%,99.0%,66.6%,0.0,114.3
4,NaN,153.0,Fijo,1.00,21.2%,7.8,21.50,36%,TO26,143.5%,99.0%,44.6%,0.0,101.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,NaN,NaN,24560.00,909.63,NaN,NaN,NaN,NaN,NaN,868.15,NaN,NaN,↓-4.56%,NaN
112,NaN,NaN,29300.00,849.28,NaN,NaN,NaN,NaN,NaN,846.82,NaN,NaN,↓-0.29%,NaN
113,NaN,NaN,25500.00,953.27,NaN,NaN,NaN,NaN,NaN,860.90,NaN,NaN,↓-9.69%,NaN
114,NaN,NaN,NaN,92.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df["indice"].unique()

array(['Badlar', 'Fijo', 'CER', 'USS', '48hs', 'USDL', '22021.50',
       '22901.00', '21670.00', '23770.00', '22890.00', '25811.00',
       '26700.00', '24560.00', '29300.00', '25500.00', nan, '90'],
      dtype=object)

In [12]:
def normalize_percentage(value):
    """
    Convierte un valor de porcentaje en formato string a un decimal.
    Ejemplo: '99.0%' se convierte en 0.99.
    Maneja los valores no numéricos o faltantes adecuadamente.
    """
    try:
        if isinstance(value, str):
            return float(value.strip('%')) / 100
    except ValueError:
        # Retorna None o un valor predeterminado en caso de valores no convertibles
        return None
    return value

In [14]:
def remove_arrows(value):
    """
    Elimina los símbolos de flecha (↑ y ↓) de un string.
    """
    if isinstance(value, str):
        return value.replace('↑', '').replace('↓', '')
    return value

In [15]:
def clean_and_convert_data(df):
    """
    Limpia y convierte los datos del DataFrame.
    """
    cleaned_df = df.copy()

    # Columnas que necesitan normalización de porcentajes
    percentage_columns = ['diferencia', 'paridad', 'qp', 'ttir', 'uptir']

    # Limpiar y convertir porcentajes
    for col in percentage_columns:
        cleaned_df[col] = cleaned_df[col].apply(remove_arrows).apply(normalize_percentage)

    # Convertir volumen y vt a numérico
    cleaned_df['volumen'] = pd.to_numeric(cleaned_df['volumen'], errors='coerce')
    cleaned_df['vt'] = pd.to_numeric(cleaned_df['vt'], errors='coerce')

    return cleaned_df

In [22]:
def percentage_nan(df):
    """
    Calcula el porcentaje de valores NaN en cada columna de un DataFrame.

    :param df: DataFrame de pandas.
    :return: Un DataFrame con el porcentaje de NaN en cada columna.
    """
    # Calcula el total de NaN en cada columna
    total_nan = df.isna().sum()

    # Calcula el porcentaje
    percentage = (total_nan / len(df)) * 100

    return percentage


In [16]:
cleaned_data = clean_and_convert_data(df)
cleaned_data.head()


,diferencia,dq,indice,md,paridad,pq,precio,qp,ticker,tir,ttir,uptir,volumen,vt
0,NaN,43.0,Badlar,0.19,0.972,31.8,115.20,0.28,BDC24,224.3%,0.99,0.158,0.0,118.5
1,NaN,63.0,Badlar,0.25,1.058,33.3,118.80,0.28,TB24,161.2%,0.99,0.100,0.0,112.3
2,NaN,92.0,Badlar,0.23,0.897,34.3,91.39,0.37,PBY24,315.9%,0.99,0.365,0.0,101.9
3,NaN,57.0,Badlar,0.38,0.845,32.8,96.50,0.34,PBA25,295.0%,0.99,0.666,0.0,114.3
4,NaN,153.0,Fijo,1.00,0.212,7.8,21.50,0.36,TO26,143.5%,0.99,0.446,0.0,101.5


¿Como imputar los valores  quedarían en NaN?


In [21]:
df_2 = cleaned_data.drop("diferencia", axis=1)
df_2

,dq,indice,md,paridad,pq,precio,qp,ticker,tir,ttir,uptir,volumen,vt
0,43.0,Badlar,0.19,0.972,31.8,115.20,0.28,BDC24,224.3%,0.99,0.158,0.0,118.5
1,63.0,Badlar,0.25,1.058,33.3,118.80,0.28,TB24,161.2%,0.99,0.100,0.0,112.3
2,92.0,Badlar,0.23,0.897,34.3,91.39,0.37,PBY24,315.9%,0.99,0.365,0.0,101.9
3,57.0,Badlar,0.38,0.845,32.8,96.50,0.34,PBA25,295.0%,0.99,0.666,0.0,114.3
4,153.0,Fijo,1.00,0.212,7.8,21.50,0.36,TO26,143.5%,0.99,0.446,0.0,101.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,NaN,24560.00,909.63,NaN,NaN,NaN,NaN,NaN,868.15,NaN,NaN,NaN,NaN
112,NaN,29300.00,849.28,NaN,NaN,NaN,NaN,NaN,846.82,NaN,NaN,NaN,NaN
113,NaN,25500.00,953.27,NaN,NaN,NaN,NaN,NaN,860.90,NaN,NaN,NaN,NaN
114,NaN,NaN,92.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
percentage_nan(df_2)

dq         18.965517
indice      0.862069
md          0.862069
paridad    10.344828
pq         10.344828
precio     18.965517
qp         27.586207
ticker     10.344828
tir         0.862069
ttir       27.586207
uptir      27.586207
volumen    18.965517
vt         18.965517
dtype: float64


Creo que una vez cruado así los datos ya podrímamos usarlo.